In [8]:
import pandas as pd
import re

In [2]:
from database import TweetDB

In [3]:
tweetdb = TweetDB()

In [4]:
engine = tweetdb.get_engine()
conn = engine.connect()

In [5]:
df= pd.read_sql('select * from tweets',conn)

In [6]:
df.head()

,id,author,tweet,timestamp
0,23114765240901633,CNNbusiness_,Oil and gold back off as dollar gains ground -...,2011-01-06 20:32:44
1,23129329785372673,CNNbusiness_,"Verizon CEO stays silent on iPhone - Sorry, Ap...",2011-01-06 21:30:36
2,23161287181598721,CNNbusiness_,Facebook likely to go public in 2012 - report ...,2011-01-06 23:37:35
3,23176209307275264,CNNbusiness_,Rent costs half my income - Amy Merrick is a f...,2011-01-07 00:36:53
4,23264239858753536,CNNbusiness_,"Crops, coal and steel inundated by Australia's...",2011-01-07 06:26:41


In [7]:
link_pattern = [r'https://t.co/[a-zA-Z0-9]*\s',r'\s+https://t.co/[a-zA-Z0-9]*\s',r'\s+https://t.co/[a-zA-Z0-9]*$']

In [12]:
def cleaner(raw):
    link_pattern = [r'https://t.co/[a-zA-Z0-9]*\s',r'\s+https://t.co/[a-zA-Z0-9]*\s',r'\s+https://t.co/[a-zA-Z0-9]*$']
    clean = raw.lower().strip()
    for ptn in link_pattern:
        clean = re.sub(ptn,'',raw)

In [13]:
df['tweet'][0]

'Oil and gold back off as dollar gains ground - Commodities started the year on a high note, with oil above $90 a bar... http://ow.ly/1aKT2m'

In [14]:
df['tweet_clean'] = df['tweet'].apply(lambda t:cleaner(t))

In [15]:
df['tweet_clean'][0]